## Links
- Original data: https://huggingface.co/datasets/timdettmers/openassistant-guanaco?row=0

- Reformat Data 1K sample: https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k?row=2

- Complete Reformat Data: https://huggingface.co/datasets/mlabonne/guanaco-llama2

- How to Prepare reformat data: https://colab.research.google.com/drive/1Ad7a9zMmkxuXTOh1Z7-rNSICA4dybpM2?usp=sharing

- How to push data to hub: https://huggingface.co/docs/datasets/v1.16.0/upload_dataset.html#:~:text=Upload%20your%20files,lines%2C%20text%2C%20and%20Parquet.

https://huggingface.co/docs/datasets/v1.16.0/upload_dataset.html

### How to fine tune Llama 2

- Free Google Colab offers a 15GB Graphics Card (Limited Resources --> Barely enough to store Llama 2–7b’s weights)

- Full fine-tuning is not possible here: we need parameter-efficient fine-tuning (PEFT) techniques like LoRA or QLoRA.

- To drastically reduce the VRAM usage, we must fine-tune the model in 4-bit precision, which is why we’ll use LoRA here.

## Import Libraries

In [1]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)

from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## Set some Params as config

In [2]:
# The model that you want to train from the Hugging Face hub
# model_name = "NousResearch/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-new-finetune"



##################
# LoRA parameters
##################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1



##############################
# bitsandbytes parameters
###############################

# Activate 4-bit precision base model loading
use_4bit = True


# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"


# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"


# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


##################################
# TrainingArguments parameters
##################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"


# Number of training epochs
num_train_epochs = 2


# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False


# Batch size per GPU for training
per_device_train_batch_size = 4



# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25


#######################
# SFT parameters
#######################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}


## Load everything and start the fine-tuning process

In [3]:
# Load Dataset
dataset = load_dataset(dataset_name, split="train")

In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [5]:
dataset[4]

{'text': '<s>[INST] Cual es el desarrollo motor de niño/a de 0 a 6 años [/INST] El desarrollo motor de un niño/a de 0 a 6 años puede ser dividido en varios hitos importantes, que suceden en un orden predecible pero que pueden variar en el tiempo que tardan en alcanzarse. Algunos de estos hitos incluyen:\n\nDesde el nacimiento hasta los 3 meses: Los bebés tienen movimientos reflejos y pueden levantar la cabeza brevemente cuando están boca abajo. Comienzan a seguir objetos con la mirada y pueden empezar a sostener objetos con las manos.\nDe 3 a 6 meses: Los bebés empiezan a rodar de lado a lado y a sentarse con apoyo. También pueden empezar a gatear y a hacer movimientos más coordinados con las manos.\nDe 6 a 9 meses: Los bebés empiezan a sentarse sin apoyo y pueden comenzar a gatear hacia adelante. También pueden sostener objetos con la mano y pueden empezar a transferir objetos de una mano a otra.\nDe 9 a 12 meses: Los bebés pueden empezar a caminar con apoyo y a levantarse sin ayuda. 

In [6]:
# Load tokenizer and model with LoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype

torch.float16

In [7]:
# load bits and bytes configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
bnb_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [8]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [9]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [11]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [12]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [13]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\trl\trainer\sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
c:\Users\sahil\.conda\envs\genai\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
# Train model
trainer.train()

  0%|          | 0/500 [00:00<?, ?it/s]

c:\Users\sahil\.conda\envs\genai\Lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
model

In [ ]:
# Save trained model
trainer.model.save_pretrained(new_model)